# Import Library

In [1]:
import pandas as pd
import numpy as np

import selenium
from selenium import webdriver

from bs4 import BeautifulSoup as bs
from urllib.request import urlopen, Request
from urllib.parse import quote_plus

import time
from datetime import datetime
from tqdm import tqdm_notebook
import warnings
warnings.filterwarnings('ignore')

# Create Empty List & Parameter Setting

In [156]:
url = 'http://www.instagram.com'
path = 'C:\\Users\\rectworks\\Downloads\\chromedriver_win32\\chromedriver.exe'

SCROLL_PAUSE_TIME = 3.0
ID = 'hanshow113'
PW = 'wlsldjtm12!'

post_link = []
popularPost_len = []

id_list = []
like_list = []
tag_list = []
link_list = []
date_list = []
month_list = []
day_list = []


# Define Function

In [9]:
def instagram_login(id, pw):
    driver.get(url)
    driver.implicitly_wait(5)
    driver.find_element_by_name('username').send_keys(id)    # id 입력
    elem_pw = driver.find_element_by_name('password')    # pw 입력
    elem_pw.send_keys(pw)
    elem_pw.submit()
    
    driver.implicitly_wait(5)    # 파싱될 때까지 5초 기다림 (미리 완료되면 waiting 종료됨)
    driver.find_element_by_class_name('cmbtv').click()    # 비밀번호 저장하지 않음
    
    driver.implicitly_wait(5)
    driver.find_element_by_xpath('/html/body/div[4]/div/div/div/div[3]/button[2]').click()    # 알림설정 무시

In [10]:
def main_search(keyword):
    search = driver.find_element_by_xpath('//*[@id="react-root"]/section/nav/div[2]/div/div/div[2]/input')
    search.clear()
    search.send_keys(keyword)
    search_list1 = driver.find_element_by_xpath('//*[@id="react-root"]/section/nav/div[2]/div/div/div[2]/div[3]/div[2]/div/a[1]')
    search_list1.click()

In [190]:
def scroll_page():
    post_link.clear()
    popularPost_len.clear()
    while True:
        pageString = driver.page_source   # page_source : 현재 렌더링된 페이지의 Elements를 모두 가져옴
        bsObj = bs(pageString, 'lxml')

        for postline in bsObj.find_all(name='div', attrs={"class":"Nnq7C weEfm"}):
            a_len = len(postline.select('a'))
            popularPost_len.append(a_len)
            # 인스타그램 게시물은 행별로 최대 3개까지 확인할 수 있는데, 최근게시물이나 마지막 게시물은 1,2개가 나올 수도 있어서 len 지정
            for post in range(a_len):   
                item = postline.select('a')[post]
                link = item.attrs['href']
                if link not in post_link:   # 스크롤을 내리고 중복된 것을 제거하지 않고 누적시키기 때문에 없는 것만 추가
                    post_link.append(link)

        last_height = driver.execute_script('return document.body.scrollHeight')   # 자바스크림트로 스크롤 길이를 넘겨줌
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")   # selenium에서 scroll 기능 사용
        time.sleep(SCROLL_PAUSE_TIME)
        # 프로세스 자체를 지정시간동안 기다려줌(무조건 지연)
        #driver.implicitly_wait(SCROLL_PAUSE_TIME)
        # 브라우저 엔진에서 파싱되는 시간을 기다려줌(요소가 존재하면 지연없이 코드 실행)
        new_height = driver.execute_script("return document.body.scrollHeight")

        if new_height == last_height:
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(SCROLL_PAUSE_TIME)
            #driver.implicitly_wait(SCROLL_PAUSE_TIME)
            new_height = driver.execute_script("return document.body.scrollHeight")

            if new_height == last_height:
                break
            else:
                last_height = new_height
                continue

In [67]:
def popularPostSum():
    popularPost = 0
    for i in range(3):
        popularPost += popularPost_len[i]
    return popularPost

# 인기게시물은 시간순으로 나타나는 것이 아니므로
# 여기에 나타나는 항목들은 시간순으로 데이터프레임 만다는 데서 제외시키기 위한 함수

In [207]:
date_string = '2020.07.01'
date =datetime.strptime(date_string, "%Y.%m.%d")
time_tuple = date.timetuple()
std_timestamp = time.mktime(time_tuple)
int(std_timestamp)

1593529200

In [231]:
def date_based_crawling(std_date):
    id_list.clear()
    like_list.clear()
    tag_list.clear()
    link_list.clear()
    date_list.clear()
    month_list.clear()
    day_list.clear()
    
    
    post_linkx = post_link[popularPostSum():]    # 스크롤 내리면서 모은 데이터 중 인기 게시물은 제외시킴
    num_of_postx = len(post_linkx)
    
    date_string = std_date
    date =datetime.strptime(date_string, "%Y.%m.%d")
    time_tuple = date.timetuple()
    std_timestamp = int(time.mktime(time_tuple))
    # 기준 날짜를 지나면 크롤링이 멈추도록 작동
    
    for i in tqdm_notebook(range(num_of_postx)):
        req = Request("https://www.instagram.com" + post_linkx[i], headers={'User-Agent': 'Mozila/5.0'})
        postpage = urlopen(req).read()
        
        req_next = Request("https://www.instagram.com" + post_linkx[i+1], headers={'User-Agent': 'Mozila/5.0'})
        postpage_next = urlopen(req_next).read()
        
        post_body = bs(postpage, 'lxml', from_encoding='utf-8')
        post_core = post_body.find('meta', attrs={'property':"og:description"})
        contents = post_core['content']

        # break할 시간 비교
        posttxt = str(postpage)    
        timestamp = int(posttxt[posttxt.find('taken_at_timestamp')+20 : posttxt.find('taken_at_timestamp')+30])
        
        posttxt_next = str(postpage_next)    
        timestamp_next = int(posttxt_next[posttxt_next.find('taken_at_timestamp')+20 : posttxt_next.find('taken_at_timestamp')+30])
        
        
        if (std_timestamp > timestamp) \
        and (datetime.fromtimestamp(timestamp).strftime('%Y/%m/%d %H:%M'))\
        == (datetime.fromtimestamp(timestamp_next).strftime('%Y/%m/%d %H:%M')): 
            break
        # 시간
        date_list.append(datetime.fromtimestamp(timestamp).strftime('%Y/%m/%d %H:%M'))
        month_list.append(datetime.fromtimestamp(timestamp).strftime("%m"))
        day_list.append(datetime.fromtimestamp(timestamp).strftime("%d"))

        # 개별 링크 리스트 
        link_list.append("https://www.instagram.com" + post_linkx[i])   
        
        # 좋아요
        try:
            likes = int(contents[: contents.find(' Likes, ')])  # Likes 문자열 앞에 있는 좋아요 개수 추출
        except:
            likes=0  # 좋아요 가 아니라 조회수로 표시되는 경우도 있어 이런 경우는 0으로 표시
        like_list.append(likes)

        # 개별 계정
        if "@" and ")" in contents:
            personal_id = contents[contents.find("@")+1 : contents.find(")")]
        elif "@" and ")" not in contents and "on Instagram" in contents:
            personal_id = contents[contents.find("@")+1 : contents.find('on Instagram')]
        else :
            personal_id = contents[1: contents.find(' posted on')]
        id_list.append(personal_id)

        '''    
        (@personal_id) on instagram, @persoanlid posted on instagram, personal_id on instgram 등의 형태로 meta 데이터에 표시되기
        때문에 여러 형식별 id 추출 if문 수행
        '''

        # 해시태그
        tag_list.append([])
        for tag_content in post_body.find_all('meta', attrs={'property':"instapp:hashtags"}):
            hashtags = tag_content['content'].rstrip(',')
            tag_list[i].append(hashtags)

# Operation check

In [72]:
driver = webdriver.Chrome(path)

instagram_login(ID, PW)

In [192]:
main_search('#춘천맛집추천')

In [193]:
scroll_page()

In [203]:
len(post_link)

690

In [205]:
post_link[0]

'/p/CDAkRsIpSUe/'

In [232]:
date_based_crawling('2020.07.01')

IndexError: list index out of range

In [233]:
insta_dict = {'날짜':date_list,'월':month_list,'일':day_list,'계정':id_list,
              '좋아요':like_list,'해시태그':tag_list,'링크':link_list}
df = pd.DataFrame(insta_dict)
df

,날짜,월,일,계정,좋아요,해시태그,링크
0,2020/07/24 15:08,07,24,신혜진 shared a post,0,"[춘천가볼만한곳, 춘천맛집추천, 춘천여행, 남춘천맛집, 춘천맛집, 홍게좋은날, 춘천...",https://www.instagram.com/p/CDA6MNunBJ4/
1,2020/07/24 14:47,07,24,아들둘맘 shared a post,0,"[춘천무한리필, 춘천무한리필맛집, 춘천맛집, 춘천고기맛집, 춘천데이트코스, 무한리필...",https://www.instagram.com/p/CDA34sVhlXP/
2,2020/07/24 13:42,07,24,cafe_almond_tree,28,[],https://www.instagram.com/p/CDAwVm4p9_R/
3,2020/07/24 13:29,07,24,kingzzang_,2,"[춘천맛집베스트, 거두리먹자골목, 거두리맛집추천, 춘천맛집추천, 킹왕짱쪽갈비춘천, ...",https://www.instagram.com/p/CDAu40UJaIz/
4,2020/07/24 13:25,07,24,kingzzang_,2,"[춘천맛집베스트, 거두리매운쪽갈비, 춘천맛집추천, 킹왕짱쪽갈비, 거두리술집, 거두리...",https://www.instagram.com/p/CDAudHwpnpg/
...,...,...,...,...,...,...,...
675,2020/07/10 15:25,07,10,cafe_almond_tree,49,"[카페추천, chuncheon, 춘천가볼만한곳, 청귤매실아이스🍐, 춘천여행, 춘천맘...",https://www.instagram.com/p/CCc5E8XpWRP/
676,2020/07/10 15:19,07,10,yujimingo,1,"[춘천맛집, 춘천여행추천, 춘천여행, 소양강, 춘천맛집황토집숯불닭갈비, 춘천닭갈비맛...",https://www.instagram.com/p/CCc4XL1Fbvz/
677,2020/07/10 14:43,07,10,ricecake_love,17,"[예쁜떡, 춘천백일케이크, 시보해제, 답례떡, 춘천도라지정과, 춘천떡주문, 상견례선...",https://www.instagram.com/p/CCc0RlEJPgg/
678,2020/07/10 14:14,07,10,top_sky888,4,"[김유정역맛집한가족숯불닭갈비, instafood, 김유정역카페, 춘천가볼만한곳, 김...",https://www.instagram.com/p/CCcw5FIFh4l/


In [218]:
post_link[393]

'/p/CA6WrWtloOS/'

In [222]:
req = Request("https://www.instagram.com" + post_link[393], headers={'User-Agent': 'Mozila/5.0'})
postpage = urlopen(req).read()

post_body = bs(postpage, 'lxml', from_encoding='utf-8')
post_core = post_body.find('meta', attrs={'property':"og:description"})
contents = post_core['content']
contents

'20 Likes, 0 Comments - 깨복이네 (@wontae1027) on Instagram: “닭갈비 에어프라이어에 돌리는 맛있네ㅎ #닭갈비#장가네춘천닭갈비#광덕계육닭갈비#춘천닭갈비맛집#에어프라이어#에어프라이어요리#전국배달#춘천닭갈비 #춘천맛집 #춘천 #춘천숯불닭갈비맛집…”'

In [223]:
post_body

<!DOCTYPE html>
<html class="no-js not-logged-in client-root" lang="en">
<head>
<meta charset="utf-8"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<title>
깨복이네 on Instagram: “닭갈비 에어프라이어에 돌리는 맛있네ㅎ #닭갈비#장가네춘천닭갈비#광덕계육닭갈비#춘천닭갈비맛집#에어프라이어#에어프라이어요리#전국배달#춘천닭갈비 #춘천맛집 #춘천 #춘천숯불닭갈비맛집 #먹스타그램 #춘천맛집추천 #춘천닭갈비맛집 #닭갈비 #춘천숯불닭갈비…”
</title>
<meta content="noimageindex, noarchive" name="robots"/>
<meta content="default" name="apple-mobile-web-app-status-bar-style"/>
<meta content="yes" name="mobile-web-app-capable"/>
<meta content="#ffffff" name="theme-color"/>
<meta content="width=device-width, initial-scale=1, minimum-scale=1, maximum-scale=1, viewport-fit=cover" id="viewport" name="viewport"/>
<link href="/data/manifest.json" rel="manifest"/>
<link as="style" crossorigin="anonymous" href="/static/bundles/metro/ConsumerUICommons.css/deb67447cbb5.css" rel="preload" type="text/css"/>
<link as="style" crossorigin="anonymous" href="/static/bundles/metro/ConsumerAsyncCommons.css/e5f471a37be6.css" 

In [224]:
posttxt = str(postpage)    
timestamp = int(posttxt[posttxt.find('taken_at_timestamp')+20 : posttxt.find('taken_at_timestamp')+30])
timestamp

1591055975

In [155]:
datetime.fromtimestamp(timestamp).strftime('%Y/%m/%d %H:%M')

'2020/05/08 10:04'

In [225]:
t1 = 1593529200
t2 = 1591055975

In [228]:
a1 = datetime.fromtimestamp(t1).strftime('%Y/%m/%d %H:%M')

In [229]:
a2 = datetime.fromtimestamp(t2).strftime('%Y/%m/%d %H:%M')

In [230]:
a1 == a2

False

In [182]:
a = ['a','b','c']
a

['a', 'b', 'c']